# AutoGluon Fit Backend Tutorial

## Prerequisite
Please read **autogluon_demo Tutorial** first, if not yet.

## Install AutoGluon

```
cd AutoGluon && python setup.py install
```

## Understanding Task, Resource and Scheduler

![Overlook](img/scheduler.png)

Resources contains number of cpus and gpus. Task includes execute function, args and its resource. TaskScheduler automatically request resource for tasks and execut it as soon as its resource is ready.

In [1]:
from autogluon.resource import Resources
from autogluon.scheduler import Task, TaskScheduler
import time

**Define custimized task**

In [2]:
def my_task():
    print('running, my_task')
    time.sleep(3.0)


**Create scheduler**

In [3]:
scheduler = TaskScheduler()

Add tasks with their resources to scheduler

In [4]:
for i in range(5):
    resource = Resources(num_cpus=2, num_gpus=0)
    task = Task(my_task, {}, resource)
    scheduler.add_task(task)

running, my_task
running, my_task
running, my_task
running, my_task
running, my_task


## Launch Task with Different Configurations

In [5]:
import autogluon as ag
from autogluon import autogluon_method

**Configuration handled by `autogluon_method`**

In [6]:
@autogluon_method
def train_fn(args):
    print('lr is {}'.format(args.lr))
    
import argparse
import numpy as np
args = argparse.ArgumentParser()
config = {'lr': ag.searcher.sample_from(
          lambda: np.power(10.0, np.random.uniform(-4, -1)))}

**Schedule Tasks**

In [7]:
myscheduler = TaskScheduler()
for i in range(5):
    resource = Resources(num_cpus=2, num_gpus=0)
    task = Task(train_fn, {'args': args, 'config': config}, resource)
    myscheduler.add_task(task)

lr is 0.001782649715138694
lr is 0.00203228544324115
lr is 0.0044915238251379995
lr is 0.07963234721775593
lr is 0.00046342505832948756


## Search Algorithm and Trial Scheduler

Search Algorithm observes the performance of each task and suggests set of hyper parameters for each new task based on the history information.
![Searcher](img/scheduler_searcher.png)
**Construct `autogluon_method` with reporter**

In [8]:
@autogluon_method
def train_fn(args, reporter):
    for e in range(10):
        # forward, backward, optimizer step and evaluation metric
        # generate fake top1_accuracy
        top1_accuracy = 1 - np.power(1.8, -np.random.uniform(e, 2*e))
        reporter(epoch=e, accuracy=top1_accuracy)

**Create hyperparameter configuration space and construct `searcher` from the configuration space**

In [9]:
import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH

cs = CS.ConfigurationSpace()
lr = CSH.UniformFloatHyperparameter('lr', lower=1e-4, upper=1e-1, log=True)
cs.add_hyperparameter(lr)

searcher = ag.searcher.RandomSampling(cs)

**Use Hyperband Scheduler or FIFO scheduler**

Scheduler monitors the training progress of each task and apply early termination if needed.

In [10]:
myscheduler = ag.scheduler.Hyperband_Scheduler(train_fn, args,
                                               resource={'num_cpus': 2, 'num_gpus': 0}, 
                                               searcher=searcher, num_trials=20,
                                               reward_attr='accuracy',
                                               time_attr='epoch',
                                               grace_period=1)
# run tasks
myscheduler.run()

**Join Tasks and Plot Results**

In [11]:
myscheduler.join_tasks()

You may get Tensorboard Visualization:
``tensorboard --logdir=./tutorials/exp/checkerpoint/logs/ --host=127.0.0.1 --port=8887``
![Backend Demo](img/backend_demo.jpg)